# Single Inference

> Warning: The major caveat here is that this really should be run via script and **NOT** in a notebook. That being said, this is the best way to introduce the concepts here.

In [1]:
import matplotlib 
# This can also be ipython magic such as `matplotlib inline` or `matplotlib notebook`
matplotlib.use('Agg')

import os,commands,signal,sys,tempfile
os.environ['GLOG_minloglevel'] = '2' # set message level to warning
import pandas
import caffe
import numpy as np
import ROOT
import lmdb
import time
#imports the larcv tools from ROOT
from larcv import larcv
import matplotlib.pyplot as plt

Welcome to ROOTaaS 6.06/08


The below instantiates the network based off the description in the prototxt and the saved pre-trained snapshot.

> NOTE: The prototxt and cfg files will need to be adjusted to reflect the local file system.
> Namely,  in `/workspace/plainresnet10b/sp_plainresnet10b_test.prototxt`, the following line should read:
> ~~~
> filler_config: "/data/plainresnet10b/test_filler.cfg"
> ~~~
> In addition, in `/workspace/plainresnet10b/test_filler.cfg`, the following line should now read: 

In [2]:
prototxt = '/workspace/plainresnet10b/sp_plainresnet10b_test.prototxt'
model_snapshot = '/workspace/plainresnet10b/sp_plainresnet10b_iter_195750.caffemodel.h5'

net = caffe.Net(prototxt, model_snapshot,caffe.TEST)

    [NORMAL]  <DataFillerProcessDriver::configure> Instantiating Process ID=0 Type: ADCThreshold w/ Name: ADCThres
    [NORMAL]  <DataFillerProcessDriver::configure> Instantiating Process ID=1 Type: SimpleFiller w/ Name: SimpleFiller
    [NORMAL]  <DataFillerIOManager::prepare_input> Opening a file in READ mode: /workspace/test.root
    [NORMAL]  <DataFillerIOManager::initialize> Prepared input with 42500 entries...


E0209 21:46:46.730245  1638 common.cpp:113] Cannot create Cublas handle. Cublas won't be available.
E0209 21:46:46.731740  1638 common.cpp:120] Cannot create Curand generator. Curand won't be available.


Now, we need to setup the IO so that we can access the data inline with caffe. This will allow us to compare prediction with output.

In [4]:
filler_name = 'DataFiller'
# check if larcv IO processor does in fact exist and registered in a factory
if not larcv.ThreadFillerFactory.exist_filler(filler_name):
    print '\033[93mFiller',filler_name,'does not exist...\033[00m'

# get IO instance, ThreadDatumFiller instance, from the factory
filler = larcv.ThreadFillerFactory.get_filler(filler_name)
filler.pd().random_access(False)
# get num events to be processed 
num_events = filler.get_n_entries()

# construct our own IO to fetch ROI object for physics analysis, use RED mode w/ same input files
myio = larcv.IOManager(0,"AnaIO")
myio.add_in_file('../test.root')
myio.initialize()

True

    [NORMAL]  <AnaIO::prepare_input> Opening a file in READ mode: ../test.root
    [NORMAL]  <AnaIO::initialize> Prepared input with 42500 entries...


## Running Inference

Here is the inference step. We're going to tell caffe to process the next batch of images. Then we're going to setup some variables to point at the relevant data.

> WARNING WARNING WARNING: Failing to adjust the mini-batch size will cause this to fail. Open up:
> `/workspace/plainresnet10b/sp_plainresnet10b_test.prototxt`
> And change layer.root_data_param.batch_size to something small. The value, `1` will work.

In [8]:
# force the filler to move the next event-to-read pointer to the entry of our interest
filler.set_next_index(0)

num_entries=10
# This will take some time. Esp w/o a GPU
net.forward()

# Wait until the filler is done filling the buffer
while filler.thread_running():
    time.sleep(0.001)

# get a vector of integers that record TTree entry numbers processed in this mini-batch
entries = filler.processed_entries()

# retrieve data already read-and-stored-in-memory from caffe blob
adcimgs = net.blobs["data"].data    # this is image
labels  = net.blobs["label"].data   # this is label
scores  = net.blobs["softmax"].data # this is final output softmax vector
            

## Processing the output

For this next part, I'm not doing much in the way of processing. Usually, we'll save some results to CSV to compare later. For now, this just prints the information as it's coming out of the mini-batch.

In [24]:
# loop over entry of mini-batch outcome
for index in xrange(1):
    print "Entry: ", entries[index]
    print "npx: ", (adcimgs > 0).sum()
    print "Label: ", int(labels)
    print "Prediction: ", scores.argmax()
    print "EMinus: ", scores[index][0]
    print "Gamma: ", scores[index][1]
    print "MuMinus: ", scores[index][2]
    print "PiMinus: ", scores[index][3]
    print "Proton: ", scores[index][4]

    if int(labels) == scores.argmax():
        print "FOUND CORRECT PARTICLE"
                
    # now get ROI data from myroi, our separate IO handle, to record physics parameters
    myio.read_entry(entries[index])
    event_roi = myio.get_data(1,'tpc_hires_crop')
                
    # loop over ROIs
    for roi in event_roi.ROIArray():
        if roi.MCSTIndex() == larcv.kINVALID_USHORT:
            print "Energy Start: ", roi.EnergyInit(), ": ", roi.EnergyDeposit()
            #print "Mass: ", larcv.larcv.ParticleMass(roi.PdgCode())
            print "Momentum At Start: ", (roi.Px(),roi.Py(),roi.Pz())
            print "Decay?: ", (np.abs(roi.PdgCode()) == 13 and np.abs(roi.ParentPdgCode()))


 Entry:  8898
npx:  308
Label:  1
Prediction:  3
EMinus:  0.151322
Gamma:  0.253516
MuMinus:  0.0832253
PiMinus:  0.502918
Proton:  0.00901945
Energy Start:  659.822443128 :  654.115157533
Momentum At Start:  (200.27086237530062, 386.0779576431554, 496.18650602415687)
Decay?:  False


## CLEANUP

More important than anything else.

In [25]:
myio.finalize()
# destroy thread filler via factory, an owner
larcv.ThreadFillerFactory.destroy_filler(filler_name)